In [2]:
import pandas as pd
from collections import defaultdict
import re
import numpy as np

filter_path = "../filter"

__ACHTUNG__: dieses Notebook dient nur dem Testen der Skripte. Es wird nicht beim Ausführen der Pipeline mit `dvc repro` ausgeführt. Alle Änderungen müssen in `skripte/abzug.py` eingefügt werden.

In [3]:
## allgemeine Funktionen


def make_nested_frame(dict) -> pd.DataFrame:
    """
    Die Funktion nimmt ein dreifach verschachteltes Dict und generiert daraus ein DataFrame mit dreifachem index
    """
    return pd.DataFrame.from_dict(
        {
            (k1, k2, k3): v3
            for k1, v1 in dict.items()
            for k2, v2 in v1.items()
            for k3, v3 in v2.items()
        },
        orient="index",
        dtype="string",
    )


def feldauswertung(kategorie: str, inhalt: str) -> list[tuple[str, str]]:
    """
    Die Funktion hat zwei Argumente: die Pica-Kategorie und den Feldinhalt.
    Per regex wird das gewünschte Unterfeld gesucht und in einem tuple mit der Klarfeldbezeichnung ausgegeben.
    Weil auch Unterfelder wiederholbar sind, kommt eine Liste von Tupeln zurück.
    """
    ergebnisse = list()
    matchlist = [
        ("209A", "f", "standort"),
        ("209A", "g", "signatur_g"),
        ("209A", "d", "ausleihcode"),
        ("209A", "a", "signatur_a"),
        ("209A", "c", "sig_komm"),
        ("209C", "a", "akz"),
        ("237A", "a", "f4801_a"),
        ("237A", "k", "f4801_k"),
        ("247C", "9", "bibliothek"),
    ]
    # [('standort', 'DBSM/M/Klemm'), ('signatur_g', 'II 1,2a - Fragm.')]
    for match in matchlist:
        if (kategorie == match[0]):
            if len(unterfeldsuche := re.findall(f"\${match[1]}([^\$]+)", inhalt)) > 0:
                if (kategorie == '209A') and (re.findall(r"\$a.+\$x0[1-8]", inhalt)):
                    pass
                else:
                    unterfeld = (match[2], "; ".join(unterfeldsuche))
                    ergebnisse.append(
                        unterfeld
                    )

    return ergebnisse


def get_exemplare(datei: str) -> pd.DataFrame:
    """
    Auf IDN und Exemplardatenfelder reduzierter Dump, dessen Dateiname als Argument übergeben wird, wird in df mit gewünschten Feldern umgewandelt
    """
    with open(f"{filter_path}/{datei}", "r") as f:
        df = pd.DataFrame(dtype="string")
        for l in f:
            # Zeile wird getrennt in Kategorie und Inhalt, line[0] ist Kategorie, line[1] ist Inhalt
            line = l.split(" ", 1)
            # wenn Leerzeile ist Datensatz zu Ende und alle Exemplare dieses Titels werden zunächst in ein df umgewandelt und dann an das df mit allen Exemplaren angehängt
            if l == "\n":
                df = pd.concat([df, make_nested_frame(exemplare)])

            # wenn beginn neuer titelsatz wird 3-fach verschachteltes dict exemplare neu angelegt und idn geschrieben
            elif line[0] == "003@":
                exemplare = defaultdict(
                    lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(str)))
                )
                idn = line[1][2:].strip()
            # wenn neue Einrichtung wird das geschrieben
            elif line[0] == "101@":
                einrichtung = line[1][2:].strip()

            # in allen anderen fällen werden die zeilen ausgelesen
            else:
                # kategorie und okkurrenz werden getrennt
                feld = line[0].split("/")
                # die unterfelder werden in klartext übersetzt
                feldergebnis = feldauswertung(feld[0], line[1].strip())
                for ergebnis in feldergebnis:
                    # wenn feld und unterfeld gleich sind, werden die ergebnisse durch Semikolon getrennt aneinander gehängt; in der funktion feldauswertung wird ausgeschlossen dass 209A/*.a zurückgegeben wird, wenn $x01-08 ist (damit werden altsignaturen aus 7101 ausgeschlossen)
                    exemplare[idn][einrichtung][feld[1]][ergebnis[0]] = "; ".join(
                        [ergebnis[1], exemplare[idn][einrichtung][feld[1]][ergebnis[0]]]
                    )

        df.index.set_names(["idn", "einrichtung", "exemplar"], inplace=True)
        return (
            df.reset_index()
            .apply(lambda x: x.str.strip("; ") if x.dtype == "string" else x)
            .astype("string")
        )


def get_titel(datei: str) -> pd.DataFrame:
    df = pd.read_csv(f"{filter_path}/{datei}", low_memory=False, dtype="string")
    df["titel"] = df.tit_a.str.cat(df.tit_d, sep=" : ", na_rep="")
    df.loc[pd.notna(df.tit_Y), "titel"] = df.tit_Y
    df.titel = df.titel.str.slice(0, 150)

    df = df.merge(
        df.fillna("").groupby(["idn"])["f4243"].apply(", ".join).reset_index(),
        on="idn",
        how="right",
    ).drop(["f4243_x"], axis=1)
    df = df.merge(
        df.fillna("").groupby(["idn"])["stuecktitel_l"].apply(", ".join).reset_index(),
        on="idn",
        how="right",
    ).drop(["stuecktitel_l_x"], axis=1)
    df = df.merge(
        df.fillna("").groupby(["idn"])["stuecktitel_a"].apply(", ".join).reset_index(),
        on="idn",
        how="right",
    ).drop(["stuecktitel_a_x"], axis=1)

    df["stuecktitel"] = df.stuecktitel_l_y.str.cat(
        df.stuecktitel_a_y, sep=" : ", na_rep=""
    )
    df.drop(
        ["stuecktitel_l_y", "stuecktitel_a_y", "tit_a", "tit_d", "tit_Y"],
        axis=1,
        inplace=True,
    )
    df.rename({"f4243_y": "f4243"}, axis=1, inplace=True)
    df.drop_duplicates(subset=["idn"], inplace=True)

    return df.astype("string")

In [24]:
# 209A/02 $fDBSM/M/Klemm$gII 1,2a - Fragm.$x09
# $aII 18,1c$x00
# $a7.a.5$x01
feldauswertung('209A', "$a7.a.5$x01".strip())

[]

In [11]:
# Bö M

titel = get_titel("böm-titel.csv")
exemplare = get_exemplare("böm-exemplare.dat")

exemplare = exemplare[
    ((exemplare.bibliothek == "009030115") | (pd.isna(exemplare.bibliothek)))
    & exemplare.signatur_a.str.startswith("Bö")
]

df = titel.merge(exemplare, on="idn", how="right")
df = df[
    [
        "idn",
        "akz",
        "bbg",
        "standort",
        "signatur_g",
        "signatur_a",
        "titel",
        "stuecktitel",
        "f4243",
        "f4256",
        "f4241",
        "f4105_9",
        "f4105_g",
        "ausleihcode",
        "sig_komm",
        "f4801_a",
        "f4801_k",
        "bibliothek",
        "einrichtung",
        "exemplar",
    ]
]
# df.to_csv("../dat/böm.csv", index=False)
df = df[~df["signatur_a"].str.contains("angeb", na=False)]
#df = df[df["f4243"].isna()]
df

,idn,akz,bbg,standort,signatur_g,signatur_a,titel,stuecktitel,f4243,f4256,f4241,f4105_9,f4105_g,ausleihcode,sig_komm,f4801_a,f4801_k,bibliothek,einrichtung,exemplar
0,011221712,L-1764-397304277,Abvz,DBSM/M/Bö,Bö M 185,Bö M 185,Almanach des muses : ou choix des poésies fugitives,:,1046059165,<NA>,<NA>,<NA>,<NA>,,,<NA>,<NA>,009030115,1,02
1,011285222,L-1790-398550824,Abvz,DBSM/M/Bö,Bö M 199/8°,Bö M 199/8°,Der @neue teutsche Merkur : vom Jahr ...,", : ,","025904507, 1046059602",<NA>,<NA>,<NA>,<NA>,,,<NA>,<NA>,009030115,1,04
2,011910828,L-1810-180652788,Abvz,DBSM/M/Bö,Bö M 356/8°,Bö M 356/8°,Almanach de l'Université Impériale : année ..,:,,<NA>,<NA>,<NA>,<NA>,,,<NA>,<NA>,009030115,1,02
3,011961449,L-1843-181302047,Abvz,DBSM/M/Bö,Bö M 196/8°,Bö M 196/8°,"Vorwärts : Volkstaschenbuch für das Jahr ..., unter Mitwirkung mehrerer freisinniger Schriftsteller Deutschlands hrsg. von Robert Blum",:,,1176570749,<NA>,<NA>,<NA>,,,<NA>,<NA>,009030115,1,02
4,012169234,L-1810-181294605,Abvz,DBSM/M/Bö,Bö M 201/8°,Bö M 201/8°,Rheinisches Taschenbuch auf das Jahr ... :,", , : , ,","012715751, 01271576X, 1016668473",<NA>,<NA>,<NA>,<NA>,,,<NA>,<NA>,009030115,1,03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599,1209071274,L-1537-672620383,Aa,DBSM/M/Bö,Bö M 311 (Angebundenes Werk),Bö M 311 (Angebundenes Werk),"L. FLORI DE// GESTIS ROMANO//RVM LIBRI QVATVOR A MENDIS// accuratissime repurgati, unà cum adno//tationibus Ioan. Camertis...// AD HAEC, SEXTI RVFFI V",:,,<NA>,<NA>,1137971126,Qd,,,<NA>,<NA>,<NA>,1,02
603,1209515709,L-1601-67451419X,Aa,DBSM/M/Bö,Bö M 229 (Angebundenes Werk),Bö M 229 (Angebundenes Werk),Disputationum de nuptiis libri VII :,:,,<NA>,<NA>,1137971231,Qd,,,<NA>,<NA>,<NA>,1,02
605,1209694948,L-1575-674894952,Aa,DBSM/M/Bö,Bö M 390 (Angebundenes Werk),Bö M 390 (Angebundenes Werk),Nathanis Chytræi// FASTORVM// ECCLESIAE CHRI-//STIANAE// Liber Secundus.// FEBRVARIVS.// :,:,,<NA>,<NA>,1137896159,Qd,,,<NA>,<NA>,<NA>,1,02
610,1211524310,L-1526-67969241X,Aa,DBSM/M/Bö,Bö M 243 (Angebundenes Werk),Bö M 243 (Angebundenes Werk),[R#[oe]mische Key-||ser ab contrauegt/ vom|| ersten Caio Julio an vntz vff den jetzig?|| H.K.Carolum. Mit kurtzer anzey=||gung ires lebens/ dapffer th,:,,<NA>,<NA>,1138240583,Qd,,,Einige d. Medaillons koloriert,,<NA>,1,02


In [4]:
# Bö Ink
titel = get_titel("böink-titel.csv")
exemplare = get_exemplare("böink-exemplare.dat")
exemplare = exemplare[exemplare.signatur_a.str.startswith("Bö")]

df = titel.merge(exemplare, on="idn", how="right")

df = df[
    [
        "idn",
        "akz",
        "bbg",
        "standort",
        "signatur_g",
        "signatur_a",
        "titel",
        "stuecktitel",
        "f4243",
        "f4256",
        "f4241",
        "f4105_9",
        "f4105_g",
        "ausleihcode",
        "sig_komm",
        "f4801_a",
        "f4801_k",
        "einrichtung",
        "exemplar",
    ]
]

df.to_csv("../dat/böink.csv", index=False)

In [4]:
# II Inkunabeln
titel = get_titel("ii-titel.csv")

exemplare = get_exemplare("ii-exemplare.dat")
exemplare = exemplare[
    ((exemplare.bibliothek == "009030115") | (pd.isna(exemplare.bibliothek)))
    & exemplare.signatur_a.str.startswith("II ")
]

df = titel.merge(exemplare, on="idn", how="right")
df = df[
    [
        "idn",
        "akz",
        "bbg",
        "standort",
        "signatur_g",
        "signatur_a",
        "titel",
        "stuecktitel",
        "f4243",
        "f4256",
        "f4241",
        "f4105_9",
        "f4105_g",
        "ausleihcode",
        "sig_komm",
        "f4801_a",
        "f4801_k",
        "einrichtung",
        "exemplar",
    ]
]
# df.to_csv("../dat/ii.csv", index=False)
df

,idn,akz,bbg,standort,signatur_g,signatur_a,titel,stuecktitel,f4243,f4256,f4241,f4105_9,f4105_g,ausleihcode,sig_komm,f4801_a,f4801_k,einrichtung,exemplar
0,1000395898,L-1853-170562638,Aal,DBSM/M/Klemm,"II 8,2c (angebunden an)","II 8,2c (angebunden an)",Die @Druckorte des XV. Jahrhunderts nebst Angabe der Erzeugnisse ihrer erstjährigen typographischen Wirksamkeit : mit e. Anh.: Verz. d. je ersten Typo,:,,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,04
1,1001280369,L-1710-174001320,Aal,DBSM/M/Klemm,"II 18,1b (angebunden)","II 18,1b (angebunden)","De librariis et bibliopolis antiquorum disputabunt d. 19. Martii, MDCCX. h. 1. q. c. :",:,,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,02
2,1032687681,L-2013-303556,Aa,DBSM/M/Klemm,"II 82,15c","II 82,15c",Flaviano Myrmeico v. c. suo salutem :,:,,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,02
3,1036843505,L-2013-316396,Aa,DBSM/M/Klemm,"II 82,7 h","II 82,7 h",Noviciis Adolescentibvs: Ad Astronomicam rempu. capessenda[m] aditu[m] i[m]petra[n]tib[us] ... Ioannis de sacro busto sph[a]ericum opusculu[m] Georgii,:,,<NA>,<NA>,1138321850,Qd,<NA>,<NA>,"Eingekl. gedr. Zeile auf d. Frontispiz: ""1478. vicesimaquarta die mensis Junii"", dazu Auskunft Oliver Duntze, GW:""Die Zeile stammt aus: GW00763 Albertus Magnus: Secreta mulierum et virorum. Mit Expositio von Henricus de Saxonia. [Venedig: Adam von Rottweil], 24.VI.1478.""",<NA>,1,02
4,1036845559,L-1485-266425038,Aa,DBSM/M/Klemm,"II 82,7 h (angebunden)","II 82,7 h (angebunden)",Clarissimi Viri Hyginii Poeticon Astronomicon. Opvs Vtilissimum Foeliciter Incipit De mundi & sph[a]er[a]e ac utriusq[ue] p[ar]tiu[m] declaratione :,:,,<NA>,<NA>,1138321850,Qd,<NA>,<NA>,<NA>,k,1,02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,1193149541,L-1498-635470098,Aa,DBSM/M/Klemm,"II 67,3a","II 67,3a",Opera :,:,,<NA>,<NA>,<NA>,<NA>,e,<NA>,<NA>,<NA>,1,02
866,1193150035,L-1498-635470314,Aa,DBSM/M/Klemm,"II 30,17a","II 30,17a",Das @ist sant Pauls leben des ersten eynsidels. und ist auch das leben des heyligen vatters sant Anthoenigs :,:,,<NA>,<NA>,<NA>,<NA>,e,<NA>,<NA>,<NA>,1,02
867,119315037X,L-1489-635470454,Aa,DBSM/M/Klemm,"II 2,14a","II 2,14a",Die @Walfart oder bilgerung unser lieben frauwen :,:,,<NA>,<NA>,<NA>,<NA>,e,<NA>,<NA>,<NA>,1,02
868,119702610X,L-1457-64487581X,Aa,DBSM/M/Klemm,"II 1,2a - Fragm.","II 1,2a - Fragm.",Psalterium cum canticis :,:,,<NA>,<NA>,<NA>,<NA>,e,<NA>,<NA>,<NA>,1,02


In [6]:
# III (Drucke 1501-1560)
titel = get_titel("iii-titel.csv")

exemplare = get_exemplare("iii-exemplare.dat")
exemplare = exemplare[
    ((exemplare.bibliothek == "009030115") | (pd.isna(exemplare.bibliothek)))
    & exemplare.signatur_a.str.startswith("III")
]

df = titel.merge(exemplare, on="idn", how="right")
df = df[
    [
        "idn",
        "akz",
        "bbg",
        "standort",
        "signatur_g",
        "signatur_a",
        "titel",
        "stuecktitel",
        "f4243",
        "f4256",
        "f4241",
        "f4105_9",
        "f4105_g",
        "ausleihcode",
        "sig_komm",
        "f4801_a",
        "f4801_k",
        "einrichtung",
        "exemplar",
    ]
]

df.to_csv("../dat/iii.csv", index=False)

In [7]:
# IV (Drucke 1561-1800)
titel = get_titel("iv-titel.csv")

exemplare = get_exemplare("iv-exemplare.dat")
exemplare = exemplare[
    ((exemplare.bibliothek == "009030115") | (pd.isna(exemplare.bibliothek)))
    & exemplare.signatur_a.str.startswith("IV")
]

df = titel.merge(exemplare, on="idn", how="right")
df = df[
    [
        "idn",
        "akz",
        "bbg",
        "standort",
        "signatur_g",
        "signatur_a",
        "jahr",
        "titel",
        "stuecktitel",
        "f4243",
        "f4256",
        "f4241",
        "f4105_9",
        "f4105_g",
        "ausleihcode",
        "sig_komm",
        "f4801_a",
        "f4801_k",
        "bibliothek",
        "einrichtung",
        "exemplar",
    ]
]
df.jahr = df.jahr.str.replace("X", "0")
df.fillna({"jahr": "0"}, inplace=True)
df = df.astype({"jahr": "int"})
df[(df.jahr < 1831)].to_csv("../dat/iv.csv", index=False)

In [8]:
# I (Handschriften, nur vor 1831)
titel = get_titel("i-titel.csv")

exemplare = get_exemplare("i-exemplare.dat")
exemplare = exemplare[
    ((exemplare.bibliothek == "009030115") | (pd.isna(exemplare.bibliothek)))
    & exemplare.signatur_a.str.startswith("I")
]

df = titel.merge(exemplare, on="idn", how="right")
df = df[
    [
        "idn",
        "akz",
        "bbg",
        "standort",
        "signatur_g",
        "signatur_a",
        "jahr",
        "titel",
        "stuecktitel",
        "f4243",
        "f4256",
        "f4241",
        "f4105_9",
        "f4105_g",
        "ausleihcode",
        "sig_komm",
        "f4801_a",
        "f4801_k",
        "bibliothek",
        "einrichtung",
        "exemplar",
    ]
]

df.jahr = df.jahr.str.replace("X", "0")
df.fillna({"jahr": "0"}, inplace=True)
df = df.astype({"jahr": "int"})

df[(df.jahr < 1831)].to_csv("../dat/i.csv", index=False)

In [9]:
# V (nur bis EJ 1830)
titel = get_titel("v-titel.csv")

exemplare = get_exemplare("v-exemplare.dat")
exemplare = exemplare[
    ((exemplare.bibliothek == "009030115") | (pd.isna(exemplare.bibliothek)))
    & exemplare.signatur_a.str.startswith("V")
]

titel.jahr = titel.jahr.str.replace("X", "0")
titel.fillna({"jahr": "0"}, inplace=True)
titel = titel.astype({"jahr": "int"})
df = titel[titel.jahr < 1831].merge(exemplare, on="idn", how="right")
df = df[
    [
        "idn",
        "akz",
        "bbg",
        "standort",
        "signatur_g",
        "signatur_a",
        "jahr",
        "titel",
        "stuecktitel",
        "f4243",
        "f4256",
        "f4241",
        "f4105_9",
        "f4105_g",
        "ausleihcode",
        "sig_komm",
        "f4801_a",
        "f4801_k",
        "bibliothek",
        "einrichtung",
        "exemplar",
    ]
]

df[df.jahr < 1831].to_csv("../dat/v.csv", index=False)

In [10]:
# VIII (nur bis EJ 1830)

titel = get_titel("viii-titel.csv")
exemplare = get_exemplare("viii-exemplare.dat")
exemplare = exemplare[
    ((exemplare.bibliothek == "009030115") | (pd.isna(exemplare.bibliothek)))
    & exemplare.signatur_a.str.startswith("VIII")
]

titel.jahr = titel.jahr.str.replace("X", "0")
titel.fillna({"jahr": "0"}, inplace=True)
titel = titel.astype({"jahr": "int"})
df = titel[titel.jahr < 1831].merge(exemplare, on="idn", how="right")
df = df[
    [
        "idn",
        "akz",
        "bbg",
        "standort",
        "signatur_g",
        "signatur_a",
        "jahr",
        "titel",
        "stuecktitel",
        "f4243",
        "f4256",
        "f4241",
        "f4105_9",
        "f4105_g",
        "ausleihcode",
        "sig_komm",
        "f4801_a",
        "f4801_k",
        "bibliothek",
        "einrichtung",
        "exemplar",
    ]
]
df[df.jahr < 1831].to_csv("../dat/viii.csv", index=False)

In [11]:
# Bö Fachbibliothek
titel = get_titel("böf-titel.csv")

exemplare = get_exemplare("böf-exemplare.dat")
exemplare = exemplare[
    ((exemplare.bibliothek == "009030115") | (pd.isna(exemplare.bibliothek)))
    & exemplare.signatur_a.str.startswith("Bö")
]

titel.jahr = titel.jahr.str.replace("X", "0")
titel.fillna({"jahr": "0"}, inplace=True)
titel = titel.astype({"jahr": "int"})

df = titel[titel.jahr < 1831].merge(exemplare, on="idn", how="right")
df = df[
    [
        "idn",
        "akz",
        "bbg",
        "standort",
        "signatur_g",
        "signatur_a",
        "jahr",
        "titel",
        "stuecktitel",
        "f4243",
        "f4256",
        "f4241",
        "f4105_9",
        "f4105_g",
        "ausleihcode",
        "sig_komm",
        "f4801_a",
        "f4801_k",
        "bibliothek",
        "einrichtung",
        "exemplar",
    ]
]
df[df.jahr < 1831].to_csv("../dat/böf.csv", index=False)

In [12]:
# Klemm Fachbibliothek

titel = get_titel("klemmf-titel.csv")

exemplare = get_exemplare("klemmf-exemplare.dat")
exemplare = exemplare[
    ((exemplare.bibliothek == "009030115") | (pd.isna(exemplare.bibliothek)))
    & (
        exemplare.signatur_a.str.contains(r"^[A-Z][a-z].*")
        | exemplare.signatur_g.str.contains(r"^[A-Z][a-z].*")
    )
]

titel.jahr = titel.jahr.str.replace("X", "0")
titel.fillna({"jahr": "0"}, inplace=True)
titel = titel.astype({"jahr": "int"})

df = titel[titel.jahr < 1831].merge(exemplare, on="idn", how="right")
df = df[
    [
        "idn",
        "akz",
        "bbg",
        "standort",
        "signatur_g",
        "signatur_a",
        "jahr",
        "titel",
        "stuecktitel",
        "f4243",
        "f4256",
        "f4241",
        "f4105_9",
        "f4105_g",
        "ausleihcode",
        "sig_komm",
        "f4801_a",
        "f4801_k",
        "bibliothek",
        "einrichtung",
        "exemplar",
    ]
]
df[df.jahr < 1831].to_csv("../dat/klemmf.csv", index=False)

In [ ]:
# Aal mit 'in:' in 4201

titel = get_titel("Aal-4201.csv")
exemplare = pd.DataFrame()
exemplare = pd.concat(
    [
        exemplare,
        get_exemplare("böink-exemplare.dat"),
        get_exemplare("böm-exemplare.dat"),
        get_exemplare("ii-exemplare.dat"),
        get_exemplare("iii-exemplare.dat"),
        get_exemplare("iv-exemplare.dat"),
    ]
)

exemplare = exemplare[
    ((exemplare.bibliothek == "009030115") | (pd.isna(exemplare.bibliothek)))
    & (
        exemplare.signatur_a.str.startswith("Bö")
        | exemplare.signatur_a.str.startswith("II")
        | exemplare.signatur_a.str.startswith("III")
        | exemplare.signatur_a.str.startswith("IV")
    )
]

titel.jahr = titel.jahr.str.replace("X", "0")
titel.fillna({"jahr": "0"}, inplace=True)
titel = titel.astype({"jahr": "int"})

df = titel[titel.jahr < 1831].merge(exemplare, on="idn", how="right")
df = df[
    [
        "idn",
        "akz",
        "bbg",
        "standort",
        "signatur_g",
        "signatur_a",
        "jahr",
        "titel",
        "stuecktitel",
        "f4243",
        "f4256",
        "f4241",
        "f4105_9",
        "f4105_g",
        "ausleihcode",
        "sig_komm",
        "f4801_a",
        "f4801_k",
        "bibliothek",
        "einrichtung",
        "exemplar",
        "4201",
    ]
]
df

In [ ]:
# Schreibmeisterbücher

titel = get_titel("schreibmeister-titel.csv")
exemplare = get_exemplare("schreibmeister-exemplare.dat")

# exemplare = exemplare[
#    ((exemplare.bibliothek == "009030115") | (pd.isna(exemplare.bibliothek)))
# ]

titel.jahr = titel.jahr.str.replace("[xX]", "0", regex=True)
titel.jahr = titel.jahr.str.replace("[\[\]]", "", regex=True)

titel.fillna({"jahr": "0"}, inplace=True)
titel = titel.astype({"jahr": "int"})

df = titel.merge(exemplare, on="idn", how="right")
df = df[
    [
        "idn",
        "akz",
        "bbg",
        "standort",
        "signatur_g",
        "signatur_a",
        "jahr",
        "titel",
        "stuecktitel",
        "f4243",
        "f4256",
        "f4241",
        "f4105_9",
        "f4105_g",
        "ausleihcode",
        "sig_komm",
        "f4801_a",
        "f4801_k",
        "einrichtung",
        "exemplar",
    ]
]
df

In [ ]:
schreibmeister_bettina = pd.read_excel("../raw/Schreibmeister.xlsx")
# df.loc[df.idn.str.strip().isin(schreibmeister_bettina.PPN.str.strip())]
schreibmeister_bettina

In [ ]:
schreibmeister_bettina.merge(
    df, left_on="PPN", right_on="idn", how="left", indicator=True
)